# This example notebook on the usage of the lstchain_create_dl3_file Tool, does the following:
## 1. Read observed DL2 file
## 2. Use the config file to generate the event list HDUs
## 3. Add the IRFs to the event list HDUs to create the DL3 file
## 4. Index the DL3 files in the given folder

To check the simple usage of the Tool, one can do the following in a terminal - 
$ lstchain_create_dl3_file --help

or for more detailed help, 
$ lstchain_create_dl3_file --help-all

Currently the Tool requires the information of the main observed source, name and RA/Dec position in degrees, to be passed as arguments, to enter source provenance information, in the final event list HDUs.

One should use the same config file for event selection on the observed data, as used on the MC DL2 files for generating the IRFs, which are to be included in the final DL3 file.

For sample DL2 file, one can use the file produced in the earlier session or the sample file stored at `/fefs/aswg/workspace/analysis-school-2022/src_indep/` or if the IRFs are produced using standard MC files of a particular data production, one should use the standard observed data DL2 files of the same data production

For example, if one used the point gamma MC DL2 file `/fefs/aswg/data/mc/DL2/20200629_prod5_trans_80/gamma/zenith_20deg/south_pointing/20210923_v0.7.5_prod5_trans_80_dynamic_cleaning/off0.4deg/dl2_gamma_20deg_180deg_off0.4deg_20210923_v0.7.5_prod5_trans_80_dynamic_cleaning_testing.h5`,
then one should use observed data DL2 file from `/fefs/aswg/data/real/DL2/20201120/v0.7.5_test_new_calibration_tree/tailcut84_dynamic_cleaning/`

To get more explanation on the data format, check https://gamma-astro-data-formats.readthedocs.io/en/latest/index.html

In [ ]:
import numpy as np
from pathlib import Path

from traitlets.config.loader import Config

from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u

In [ ]:
from lstchain.io.io import read_data_dl2_to_QTable
from lstchain.reco.utils import get_effective_time
from lstchain.paths import run_info_from_filename, dl2_to_dl3_filename

from lstchain.irf.hdu_table import (
    create_event_list, add_icrs_position_params,
    create_hdu_index_hdu, create_obs_index_hdu
)
from lstchain.io.event_selection import EventSelector, DL3FixedCuts
from lstchain.io.config import read_configuration_file

# 1. Input parameters to convert DL2 file to DL3

In [ ]:
# Modify the paths as applicable.
base_dir = "/fefs/aswg/workspace/analysis-school-2022/"

dl2_dir = "src_indep/"

input_dl2_path = base_dir + dl2_dir + "dl2_LST-1.Run2968_first10.h5"

output_dl3_path = "./DL3_src_indep/"
Path(output_dl3_path).mkdir(exist_ok=True)

config_file = base_dir + dl2_dir + "irf_tool_config.json"

irf_file = base_dir + dl2_dir + "pnt_irf_pnt_gamma.fits.gz"

source_name = "Crab"
source_ra = "83.63308333deg"
source_dec = "22.0145deg"

overwrite = True

In [ ]:
dl3_filename = dl2_to_dl3_filename(input_dl2_path)
source_pos = SkyCoord(ra=source_ra, dec=source_dec)
print("The name of the DL3 file will be,", dl3_filename)
print("Using the source RA and DEC values to get the source position,", source_pos)

# 2. Get the information of the data DL2 file

In [ ]:
%%time
data = read_data_dl2_to_QTable(input_dl2_path)

In [ ]:
%%time
effective_time, elapsed_time = get_effective_time(data)
run_number = run_info_from_filename(input_dl2_path)[1]
print("The effective time of the run is,",effective_time, "and the total elapsed time of the run is,", elapsed_time)
print("The run number used from the DL2 file is,", run_number)

# 3. Apply selection cuts and bin the events as per the config file

In [ ]:
config = Config(read_configuration_file(config_file))
config

In [ ]:
# Using the Components for the event selection and for applying some cuts
event_sel = EventSelector(config=Config(config))
fixed_cuts = DL3FixedCuts(config=Config(config))

In [ ]:
%%time
# Applying event filters to the Hillas parameters
data_size_0 = len(data)
print("Size of the table before the filter -", len(data))
data = event_sel.filter_cut(data)
print("Size of the table after the filter -", len(data))

In [ ]:
%%time
# Applying the gammaness cut
print("Size of the table before the cut -", len(data))
data = fixed_cuts.gh_cut(data)
print("Size of the table after the cut -", len(data))

# 4. Convert the position parameters of the events in ICRS frame and add to the table

In [ ]:
%%time
data = add_icrs_position_params(data, source_pos)

# 5. Create the HDUs for the DL3 file

In [ ]:
%%time
events, gti, pointing = create_event_list(
    data, run_number, source_name, source_pos, effective_time.value, elapsed_time.value
)

# 6. Add the HDUs to the final HDUList

In [ ]:
hdulist = fits.HDUList([fits.PrimaryHDU(), events, gti, pointing])

In [ ]:
irf_hdus = fits.open(irf_file)

In [ ]:
for irf in irf_hdus[1:]:
    hdulist.append(irf)

# 7. Write the HDUList to the final DL3 file

In [ ]:
hdulist.writeto(output_dl3_path + dl3_filename, overwrite=overwrite)

# 8. Check the DL3 fits info

In [ ]:
f = fits.open(output_dl3_path + dl3_filename)
f.info()

# 9. Index the DL3 files in the folder

In [ ]:
%%time
# Get the list of all DL3 files in the given output destination
list_dl3_files = sorted(Path(output_dl3_path).glob("dl3*gz"))

file_list = []
for d in list_dl3_files:
    file_list.append(d.name)
print(file_list)

In [ ]:
%%time
create_hdu_index_hdu(
    file_list, 
    Path(output_dl3_path), 
    Path(output_dl3_path)/"hdu-index.fits.gz", 
    overwrite
)

In [ ]:
%%time
create_obs_index_hdu(
    file_list, 
    Path(output_dl3_path), 
    Path(output_dl3_path)/"obs-index.fits.gz", 
    overwrite
)

# 10. Check the DL3 Index files

In [ ]:
Table.read(output_dl3_path+"hdu-index.fits.gz")

In [ ]:
Table.read(output_dl3_path+"hdu-index.fits.gz").meta

In [ ]:
Table.read(output_dl3_path+"obs-index.fits.gz")

In [ ]:
Table.read(output_dl3_path+"obs-index.fits.gz").meta